In [14]:
import google.generativeai as genai
import os

# Configure your Gemini API key (set this securely or via environment variable)
genai.configure(api_key=os.getenv("GOOGLE_GENAI_API_KEY"))

# Initialize Gemini Pro model for chat
model = genai.GenerativeModel("gemini-2.5-flash")

def get_gemini_response(prompt):
    response = model.generate_content(prompt)
    return response.text



In [15]:
print("Gemini Pro Chatbot")
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        break
    response = get_gemini_response(user_input)
    print("Gemini:", response)

Gemini Pro Chatbot
Gemini: Hello! How can I help you today?
Gemini: To count all records in your `matches` table, use the following SQL query:

```sql
SELECT COUNT(*) FROM matches;
```
Gemini: As an AI, I don't have a program or session to "exit" from in the traditional sense.

If you're done interacting with me, you can simply:

*   Close your browser tab or window.
*   Start a new conversation (if your platform has that option).
*   Just stop typing.

I'll be here if you need anything else!


In [13]:
import google.generativeai as genai
for model in genai.list_models():
    print(model.name)

models/embedding-gecko-001
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/learnlm-2.0-flash-experimental
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-l

In [2]:
import ollama

model_name = "llama3.2:1b"  # replace with your actual model name
messages = [{"role": "system", 
    "content": "You are a sql query generator. Generate a sql query based on the user input. only return the sql query."}]

while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        break
    messages.append({"role": "user", "content": user_input})
    
    response = ollama.chat(model=model_name, messages=messages)
    bot_reply = response.message.content
    print("Bot:", bot_reply)
    
    messages.append({"role": "assistant", "content": bot_reply})


Bot: ```
SELECT * FROM users WHERE age > @age;
```


In [3]:
import os
import json
from dotenv import load_dotenv
from langchain_community.utilities import SQLDatabase
import google.generativeai as genai

load_dotenv()

# Configure Gemini API key and model
genai.configure(api_key=os.getenv("GOOGLE_GENAI_API_KEY"))
model = genai.GenerativeModel("gemini-2.5-flash")

# Helper function to send prompt to Gemini and get completion text
def get_gemini_response(prompt):
    response = model.generate_content(prompt)
    return response.text.strip()

# Load DB credentials from environment
db_user = os.getenv("db_user")
db_password = os.getenv("db_password")
db_host = os.getenv("db_host")
db_port = os.getenv("db_port")
db_name = "jiohotstar_ads"

# Setup LangChain SQLDatabase connection
db_path = f"mysql+mysqlconnector://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
db = SQLDatabase.from_uri(db_path)

# Get schema info for prompt context
schema_info = db.get_table_info()

print("Usable tables:", db.get_usable_table_names())

# Prompt template for SQL generation
sql_generation_template = """
You are an expert SQL query writer.

Database schema:
{schema}

Instructions:
- Generate a valid MySQL SELECT query.
- Use only the tables and columns present in the schema.
- If the question cannot be answered with the schema, output: SELECT 'CANNOT_ANSWER' AS error_msg;

User question:
{question}

Write ONLY the SQL query, no additional text.
"""

# Prompt template for natural language answer generation
answer_generation_template = """
You are a helpful data analyst.

Original question:
{question}

SQL query executed:
{sql}

Query result (showing first few rows):
{result}

Using ONLY this information, provide a clear and concise answer.
If the result is insufficient to answer, say you cannot answer confidently.
"""

# Interactive loop
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        break

    # Format prompt for SQL generation
    sql_prompt = sql_generation_template.format(schema=schema_info, question=user_input)

    # Generate SQL query with Gemini
    generated_sql = get_gemini_response(sql_prompt)
    print("Generated SQL:\n", generated_sql)

    # Run SQL safely (simple check)
    lowered = generated_sql.lower()
    if any(x in lowered for x in ["insert", "update", "delete", "drop", "alter"]):
        print("Dangerous SQL detected. Skipping execution.")
        continue

    try:
        results = db.run(generated_sql)
    except Exception as e:
        print("SQL execution error:", e)
        continue

    # Prepare JSON trimmed result for prompt
    result_json = json.dumps(results[:10], indent=2, default=str)

    # Format prompt for natural language answer generation
    answer_prompt = answer_generation_template.format(
        question=user_input,
        sql=generated_sql,
        result=result_json
    )

    # Generate natural language answer with Gemini
    natural_answer = get_gemini_response(answer_prompt)
    print("Bot (natural language answer):\n", natural_answer)


d:\GIT\JioHotstar-Ad-Vision\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Usable tables: ['matches']
Generated SQL:
 SELECT 'CANNOT_ANSWER' AS error_msg;
Bot (natural language answer):
 I cannot answer confidently. The executed SQL query returned an error message (`CANNOT_ANSWER`) instead of the requested database table information.
Generated SQL:
 ```sql
SELECT * FROM matches;
```
SQL execution error: (mysql.connector.errors.ProgrammingError) 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '```sql
SELECT * FROM matches;
```' at line 1
[SQL: ```sql
SELECT * FROM matches;
```]
(Background on this error at: https://sqlalche.me/e/20/f405)
